In [5]:
pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.2 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from keras.callbacks import EarlyStopping
import keras_tuner as kt
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
def round_up_metric_results(results) -> list:
    return list(map(lambda el: round(el, 3), results))

In [3]:
df = pd.read_csv("train.csv")


df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "H", "label"] = 1
df.loc[df["label"] == "L", "label"] = 2


In [5]:
with open("plank_input_scaler.pkl", "rb") as f2:
    sc = pickle.load(f2)

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

y_cat = to_categorical(y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [7]:
stop_early = EarlyStopping(monitor='loss', patience=3)


final_models = {}

In [8]:
def describe_model(model):


    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")

In [9]:
def desc_model(tuner):

    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [10]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(68, input_dim = 68, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(3, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [11]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_4'
)
tuner_7lD.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 01m 25s]
accuracy: 0.998904287815094

Best accuracy So Far: 0.9993425607681274
Total elapsed time: 00h 15m 08s


In [13]:
model_7lD = desc_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 68 units, func:  <function relu at 0x122141f70>
Layer-2: 352 units, func:  <function tanh at 0x3261d9ca0>
Layer-3: 0 units, func: None
Layer-4: 416 units, func:  <function tanh at 0x3261d9ca0>
Layer-5: 0 units, func: None
Layer-6: 64 units, func:  <function tanh at 0x3261d9ca0>
Layer-7: 3 units, func:  <function softmax at 0x3261d98b0>

Other params:
dropout_1: 0.30000000000000004
dropout_2: 0.2
learning_rate: 0.0001
Epoch 1/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 34s 14ms/step - accuracy: 0.8984 - loss: 0.2750 - val_accuracy: 0.9981 - val_loss: 0.0100
Epoch 2/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - accuracy: 0.9977 - loss: 0.0114 - val_accuracy: 0.9986 - val_loss: 0.0054
Epoch 3/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - accuracy: 0.9980 - loss: 0.0081 - val_accuracy: 0.9991 - val_loss: 0.0041
Epoch 4/100
2282/2282 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - accuracy: 0.9983 - loss: 0.0067 - val_accuracy: 0.9991 - val_loss: 0.0033
Epoch 5/1

In [15]:
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

predict_x = model_7lD.predict(x_test, verbose=False)

y_pred_class = np.argmax(predict_x, axis=1)

y_test_class = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])

p_score, r_score, f_score, _ = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])

def round_up_metric_results(scores, decimals=3):
    return [round(score, decimals) for score in scores]

results = {
    "precision": round_up_metric_results(p_score),
    "recall": round_up_metric_results(r_score),
    "f1_score": round_up_metric_results(f_score),
    "confusion_matrix": cm
}

print("Model Results:")
print(f"Precision: {results['precision']}")
print(f"Recall: {results['recall']}")
print(f"F1-Score: {results['f1_score']}")
print("Confusion Matrix:")
print(results['confusion_matrix'])

Model Results:
Precision: [0.999, 1.0, 0.999]
Recall: [0.999, 1.0, 0.999]
F1-Score: [0.999, 1.0, 0.999]
Confusion Matrix:
[[1914    0    1]
 [   0 1844    0]
 [   1    0 1944]]


In [18]:
predict_x = model_7lD.predict(test_x, verbose=False) 
y_pred_class = np.argmax(predict_x, axis=1)
y_test_class = np.argmax(test_y_cat, axis=1)

cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
(p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])

NameError: name 'test_x' is not defined

In [19]:
# load dataset
test_df = pd.read_csv("test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "H", "label"] = 1
test_df.loc[test_df["label"] == "L", "label"] = 2

print(f'Number of rows: {test_df.shape[0]} \nNumber of columns: {test_df.shape[1]}\n')
print(f"Labels: \n{test_df['label'].value_counts()}\n")

Number of rows: 710 
Number of columns: 69

Labels: 
label
1    241
2    235
0    234
Name: count, dtype: int64



In [20]:
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [21]:
predict_x = model_7lD.predict(test_x, verbose=False) 
y_pred_class = np.argmax(predict_x, axis=1)
y_test_class = np.argmax(test_y_cat, axis=1)

cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1, 2])
(p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1, 2])

In [23]:
print((round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

([0.962, 0.851, 0.851], [0.654, 0.996, 0.974], [0.779, 0.918, 0.909], array([[153,  42,  39],
       [  0, 240,   1],
       [  6,   0, 229]]))


In [25]:
with open("plank_likdp.pkl", "wb") as f:
    pickle.dump(model_7lD, f)